<a href="https://colab.research.google.com/github/laahiri2005/PRODIFY-INFO-TECH/blob/main/PRODIGY_ML_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import numpy as np
import os
import cv2
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

In [34]:
CAT_DIR = '/content/drive/MyDrive/cat/cats'
DOG_DIR = '/content/drive/MyDrive/dog/dogs'
IMG_SIZE = 64

In [35]:
def load_images_from_folder(folder, label):
    images = []
    for filename in tqdm(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append((img, label))
    return images

In [36]:
cat_images = load_images_from_folder(CAT_DIR, 0)
dog_images = load_images_from_folder(DOG_DIR, 1)

100%|██████████| 5/5 [00:00<00:00, 158.93it/s]


In [37]:
data = cat_images + dog_images
random.shuffle(data)

In [38]:
X = np.array([item[0] for item in data])
y = np.array([item[1] for item in data])

In [39]:
X = X / 255.0

X_flat = X.reshape(X.shape[0], -1)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

In [41]:
model = SVC(kernel='linear', random_state=42)
model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [42]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 50.00%


In [43]:
import joblib
joblib.dump(model , '/content/drive/MyDrive/dog_cat_classifier.pkl')

['/content/drive/MyDrive/dog_cat_classifier.pkl']

In [44]:
model = joblib.load('/content/drive/MyDrive/dog_cat_classifier.pkl')

In [45]:
def preprocess_image(image_path, img_size=64):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (img_size, img_size))
        img = img / 255.0
        img_flat = img.flatten().reshape(1, -1)
        return img_flat
    else:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

def predict_image(image_path, model):
    img_flat = preprocess_image(image_path)
    prediction = model.predict(img_flat)
    return 'Dog' if prediction == 1 else 'Cat'

In [46]:
image_path = '/content/drive/MyDrive/dog.jpg'
print(predict_image(image_path, model))

Dog
